In [1]:
pip install pandas && pip install numpy

Defaulting to user installation because normal site-packages is not writeable
zsh:1: command not found: pip
Note: you may need to restart the kernel to use updated packages.


In [2]:
pip install tweepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
#import tweepy
import requests
import os
import json
import csv
import asyncio.sleep

with open('keys.txt', 'r') as tfile:
    consumer_key = tfile.readline().strip('\n')
    consumer_secret = tfile.readline().strip('\n')
    access_token = tfile.readline().strip('\n')
    access_secret = tfile.readline().strip('\n')
    bearer_token = tfile.readline().strip('\n')


In [4]:
def bearer_oauth(r):
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2FilteredStreamPython"
    return r

In [5]:
def get_rules():
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream/rules", auth=bearer_oauth
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot get rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))
    return response.json()

In [6]:
def delete_all_rules(rules):
    if rules is None or "data" not in rules:
        return None

    ids = list(map(lambda rule: rule["id"], rules["data"]))
    payload = {"delete": {"ids": ids}}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload
    )
    if response.status_code != 200:
        raise Exception(
            "Cannot delete rules (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
    print(json.dumps(response.json()))


In [7]:
def set_rules(delete):
    sample_rules = [
        #{"value": "covid OR corona OR (vacina causa (hiv OR aids)) OR vachina"},
        {"value": "context:123.1220701888179359745", "tag": "covid-19"},
        
    ]
    payload = {"add": sample_rules}
    response = requests.post(
        "https://api.twitter.com/2/tweets/search/stream/rules",
        auth=bearer_oauth,
        json=payload,
    )
    if response.status_code != 201:
        raise Exception(
            "Cannot add rules (HTTP {}): {}".format(response.status_code, response.text)
        )
    print(json.dumps(response.json()))

In [8]:
def get_params():
    return {"tweet.fields": "lang,created_at",
            "expansions": "author_id",
           }

In [9]:
filename = "tweetsLarge"

def get_stream(set):
    response = requests.get(
        "https://api.twitter.com/2/tweets/search/stream", auth=bearer_oauth, stream=True, params=get_params())
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(
            "Cannot get stream (HTTP {}): {}".format(
                response.status_code, response.text
            )
        )
        
    #testing
    max_tweet_count = 300
    count = 1
    
    file = open(filename+".csv", 'a')
    writer = csv.writer(file)
    writer.writerow(["tweet", "tweet URL", "author id", "created at"])

    
    for response_line in response.iter_lines():
        #TODO checar e salvar tmb full text tweet (>140 char)
        if response_line:
            decoded_line = response_line.decode('utf-8')
            json_response = json.loads(decoded_line)
            
            if json_response['data']['text'].startswith('RT @'):
                continue
            if json_response['data']['lang'] != 'pt':#bug ta pegando pt portugal tmb
                continue
            #remover tweets resposta?
            #remover tweets com excesso de hashtag / emoji
            
            print(json.dumps(json_response, indent=4, sort_keys=True))
            
            #save tweet to csv
            tweetText = json_response['data']['text']
            authorId = json_response['data']['author_id']
            tweetId = json_response['data']['id']
            createdAt = json_response['data']['created_at']
            #TODO hasMedia?(links)
            #TODO usuarios mencionados?
            linkTweet = "https://twitter.com/user/status/" + str(tweetId)
           
            writer.writerow([tweetText, linkTweet, authorId, createdAt])
            
            if count > max_tweet_count:
                break
            count += 1
            
    file.close()

In [10]:
def main():
    rules = get_rules()
    delete = delete_all_rules(rules)
    set = set_rules(delete)
    
    timeout = 0
    while True:
        sleep(2**timeout)
        timeout += 1
        get_stream(set)


if __name__ == "__main__":
    main()

{"data": [{"id": "1455271508478238729", "value": "context:123.1220701888179359745", "tag": "covid-19"}], "meta": {"sent": "2021-11-01T20:37:42.901Z", "result_count": 1}}
{"meta": {"sent": "2021-11-01T20:37:44.924Z", "summary": {"deleted": 1, "not_deleted": 0}}}
{"data": [{"value": "context:123.1220701888179359745", "tag": "covid-19", "id": "1455272874730795012"}], "meta": {"sent": "2021-11-01T20:37:46.570Z", "summary": {"created": 1, "not_created": 0, "valid": 1, "invalid": 0}}}
200
{
    "data": {
        "author_id": "1431264621411938310",
        "created_at": "2021-11-01T20:37:42.000Z",
        "id": "1455272860533067776",
        "lang": "pt",
        "text": "@bozistas @UOL Vacina contra o t\u00e9tano n\u00e3o \u00e9 experimental. As vacinas contra covid conforme os pr\u00f3prios fabricantes dizem s\u00e3o EXPERIMENTAIS."
    },
    "includes": {
        "users": [
            {
                "id": "1431264621411938310",
                "name": "Eduardo",
                "usern

{
    "data": {
        "author_id": "64311101",
        "created_at": "2021-11-01T20:37:52.000Z",
        "id": "1455272902832558080",
        "lang": "pt",
        "text": "\"\u00a7 2\u00ba Considera-se pr\u00e1tica discriminat\u00f3ria a obrigatoriedade de certificado de vacina\u00e7\u00e3o em processos seletivos de admiss\u00e3o de trabalhadores, assim como a demiss\u00e3o por justa causa de empregado em raz\u00e3o da n\u00e3o apresenta\u00e7\u00e3o de certificado de vacina\u00e7\u00e3o\"\nhttps://t.co/fhzQR4yXlS https://t.co/l7R7x0IuQU"
    },
    "includes": {
        "users": [
            {
                "id": "64311101",
                "name": "Clarissa",
                "username": "clarissaganzer"
            },
            {
                "id": "44333533",
                "name": "Paula Schmitt",
                "username": "schmittpaula"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-

{
    "data": {
        "author_id": "84937026",
        "created_at": "2021-11-01T20:37:57.000Z",
        "id": "1455272924076822528",
        "lang": "pt",
        "text": "desde antes de pandemia eu comemoro qualquer coisa pra beber\nCinco de Mayo, St. Patrick'S Day, Oktoberfest... esse povo que fica cagando regra n\u00e3o vive https://t.co/t3jV48J9KW"
    },
    "includes": {
        "users": [
            {
                "id": "84937026",
                "name": "CrisTina",
                "username": "Tina_LittleCris"
            },
            {
                "id": "880944931413659649",
                "name": "Preta",
                "username": "pretademaiss"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "1055819614217736192",
        "created_at": "2021-11-01T20:37:58.000Z",
        "id": "1455272927075741712",
        "lang": "p

{
    "data": {
        "author_id": "135329284",
        "created_at": "2021-11-01T20:38:07.000Z",
        "id": "1455272968007950339",
        "lang": "pt",
        "text": "@wagn_am @o_antagonista Ent\u00e3o pra que serviu a vacina que vc tomou, s\u00f3 para tirar foto com a agente de sa\u00fade ?"
    },
    "includes": {
        "users": [
            {
                "id": "135329284",
                "name": "Orenascido_38",
                "username": "Remdec2020"
            },
            {
                "id": "111975135",
                "name": "WagnerAm \u2122 \ud83d\udc0a",
                "username": "wagn_am"
            },
            {
                "id": "2880932962",
                "name": "O Antagonista",
                "username": "o_antagonista"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "127024954",
        "c

{
    "data": {
        "author_id": "962734523329990662",
        "created_at": "2021-11-01T20:38:15.000Z",
        "id": "1455273001327501316",
        "lang": "pt",
        "text": "@VilmaReis15 @bozistas @UOL Pessoal, calma! Esse governo de merda tem apenas mais um ano pra durar...vamos aguentar firme at\u00e9 l\u00e1. (e medidas como essa de ir contra a vacina\u00e7\u00e3o serve ainda mais pra sujar a imagem desse governo que todos est\u00e3o vendo n\u00e3o ser l\u00e1 essas coisas)"
    },
    "includes": {
        "users": [
            {
                "id": "962734523329990662",
                "name": "Jo\u00e3ozito",
                "username": "kyloren_196780"
            },
            {
                "id": "860924725056479232",
                "name": "Vilma Reis -",
                "username": "VilmaReis15"
            },
            {
                "id": "1351689119752183814",
                "name": "Bozistas alvos da policia",
                "username": "bozista

...

{
    "data": {
        "author_id": "1295695418282708992",
        "created_at": "2021-11-01T20:40:31.000Z",
        "id": "1455273571299778564",
        "lang": "pt",
        "text": "CPI diz ao STF que Bolsonaro cometeu crime ao associar vacina\u00e7\u00e3o a\u00a0HIV https://t.co/qmQHy49pM8"
    },
    "includes": {
        "users": [
            {
                "id": "1295695418282708992",
                "name": "Bus\u00e3o Foz",
                "username": "BusaoFoz"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "3258790606",
        "created_at": "2021-11-01T20:40:31.000Z",
        "id": "1455273571220168707",
        "lang": "pt",
        "text": "@UOL N\u00e3o somos contra a vacina, mas, o for\u00e7amento, a obrigatoriedade \u00e9 um ato de abuso, desrespeito a liberdade de escolha ou opini\u00e3o. Tamb\u00e9m conhecemos que govern

{
    "data": {
        "author_id": "42453999",
        "created_at": "2021-11-01T20:40:34.000Z",
        "id": "1455273582951604234",
        "lang": "pt",
        "text": "Livro: Reportagem, da ditadura a pandemia. https://t.co/zH1wPZUDtm"
    },
    "includes": {
        "users": [
            {
                "id": "42453999",
                "name": "M\u00e1rcia Turcato",
                "username": "marciaturcato"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "1007233676667650048",
        "created_at": "2021-11-01T20:40:36.000Z",
        "id": "1455273590534909953",
        "lang": "pt",
        "text": "Coronav\u00edrus: com mais uma morte nesta segunda (1\u00ba), Cachoeiro chega a 640\u00a0\u00f3bitos https://t.co/jqq8whaCX7"
    },
    "includes": {
        "users": [
            {
                "id": "1007233676667650048",
     

{
    "data": {
        "author_id": "96047355",
        "created_at": "2021-11-01T20:40:41.000Z",
        "id": "1455273611892305925",
        "lang": "pt",
        "text": "Marighella x meu oretenso distanciamento social! \ud83d\ude31\ud83d\ude30\ud83d\ude30\ud83d\ude30 @ Rio de Janeiro, Rio de Janeiro https://t.co/i4JtOR5Naq"
    },
    "includes": {
        "users": [
            {
                "id": "96047355",
                "name": "Aurea Maria Xavier",
                "username": "gatosemolho"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "909570116630847488",
        "created_at": "2021-11-01T20:40:41.000Z",
        "id": "1455273611162492938",
        "lang": "pt",
        "text": "Linda mascara \ud83d\udd78\ud83d\udd77\ud83d\udc99\ud83d\ude08 https://t.co/WswSa3cypp"
    },
    "includes": {
        "users": [
            {
    

{
    "data": {
        "author_id": "1320817982",
        "created_at": "2021-11-01T20:40:50.000Z",
        "id": "1455273648818987008",
        "lang": "pt",
        "text": "literalmente https://t.co/OHXs5uLDGA"
    },
    "includes": {
        "users": [
            {
                "id": "1320817982",
                "name": "sandri",
                "username": "sandrita_f27"
            },
            {
                "id": "944793247578071040",
                "name": "Hache",
                "username": "GiskardH"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "1090610216750796801",
        "created_at": "2021-11-01T20:40:54.000Z",
        "id": "1455273666611224577",
        "lang": "pt",
        "text": "vou tomar segunda dose da vacina amanh\u00e3"
    },
    "includes": {
        "users": [
            {
                "id": "10

{
    "data": {
        "author_id": "797650380238753792",
        "created_at": "2021-11-01T20:41:03.000Z",
        "id": "1455273705957961732",
        "lang": "pt",
        "text": "**** Os pesquisadores fizeram o \u00f3bvio. Eu mesmo qdo tive covid fazia bochechos e gargarejos constantemente para diminuir a reprodu\u00e7\u00e3o viral. E era com Plax mesmo. https://t.co/9Vux8lrjrF"
    },
    "includes": {
        "users": [
            {
                "id": "797650380238753792",
                "name": "Rollo \ud83e\udd4b",
                "username": "RolloReal"
            }
        ]
    },
    "matching_rules": [
        {
            "id": "1455272874730795012",
            "tag": "covid-19"
        }
    ]
}
{
    "data": {
        "author_id": "886067723037552640",
        "created_at": "2021-11-01T20:41:06.000Z",
        "id": "1455273716024283138",
        "lang": "pt",
        "text": "Minist\u00e9rio do Trabalho publica portaria que impede demiss\u00e3o por justa causa